# GARPOS Notebook 
### This notebook demonstrates how to run the [GARPOS](https://github.com/s-watanabe-jhod/garpos) GNSS-A processing software in the EarthScope GeoLab cloud environment.  
11-07-2025

### GeoLab Basics

GeoLab is the name of EarthScope's Jupyter Hub, which allows users to run code via Jupyter notebooks (or in a terminal) on EarthScope resources in AWS.  

The `sfg-geolab` image comes pre-loaded with the software needed to process GNSS-A data. Each user has their own home directory `/home/jovyan` that is mounted to 
the image and any files kept there are accessibile only to that user and remain during future sessions.  

### Data 
Currently, we have pre-processed `cascadia-gorda` data and stored the shotdata in an S3 bucket accessible to users authorized to use the `s3-seafloor` role.  

Please contact mike.gottlieb@earthscope.org to request access to this role.

Raw data is available to anyone with an earthscope account via authenticated https from [https://data.earthscope.org/archive/seafloor](https://data.earthscope.org/archive/seafloor).

### Metadata
In order to simplify our interactions with the various metadata formats, we are translating all metadata into `site` and `vessel` json. We store and load these metadata from the S3 archive [https://data.earthscope.org/archive/seafloor/metadata](https://data.earthscope.org/archive/seafloor/metadata)

The site metadata file is stored at `/home/jovyan/[network]/[site]/[metadata]/`

### Results
Results are stored per GARPOS run under `/home/jovyan/[network]/[site]/[campaign]/[survey]/GARPOS/results`

### Logs
Some log output is piped to the notebook.  Additionally, specific survey logs are written to `/home/jovyan/[network]/[site]/[campaign]/[survey]/GARPOS/logs`

## Import Modules

In [ ]:
import os

from es_sfgtools.config.env_config import Environment
from es_sfgtools.workflows.workflow_handler import WorkflowHandler
from earthscope_sdk import AsyncEarthScopeClient
from earthscope_sdk.config.settings import SdkSettings
import logging
from es_sfgtools.logging.loggers import set_all_logger_levels
set_all_logger_levels(logging.INFO)

#this reads environment variables set in the image
Environment.load_working_environment()

## Grab temporary AWS credentials for the s3-seafloor role
For more information on the EarthScope SDK, read the [documentation](https://docs.earthscope.org/projects/SDK/en/stable/intro.html). 

If you do not have authorization for this role and think you should, email a request to mike.gottlieb@earthscope.org with your EarthScope login id.  

In [ ]:
# Create an EarthScope client 
es = AsyncEarthScopeClient()
# Set AWS credentials for S3 access
creds = await es.user.get_aws_credentials(role="s3-seafloor")

os.environ['AWS_ACCESS_KEY_ID'] = creds.aws_access_key_id
os.environ['AWS_SECRET_ACCESS_KEY'] = creds.aws_secret_access_key
os.environ['AWS_SESSION_TOKEN']  = creds.aws_session_token

## Select the data you are interested in running through GARPOS
Currently we have pre-processed all campaigns for stations **GCC1**, **NBR1**, **NCC1** and **NTH1** in network **cascadia-gorda** and made shotdata available to this environment in a shared S3 bucket.

Available Cascadia Campaigns
|        | GCC1        | NBR1        | NCC1        | NTH1        |
|--------|-------------|-------------|-------------|-------------|
| **2022** | 2022_A_1065 |             | 2022_A_1065 |             |
| **2023** | 2023_A_1063 | 2023_A_1063 | 2023_A_1063 |             |
| **2024** | 2024_A_1126 | 2024_A_1126 | 2024_A_1126 |             |
| **2025** | 2025_A_1126 | 2025_A_1126 | 2025_A_1126 | 2025_A_1126 |

#### Select the network, station and campaign (above) that you want to run in the cell below. 


In [ ]:
NETWORK = "cascadia-gorda"
STATION = "GCC1"
CAMPAIGN = "2025_A_1126"

## Set up the workflow with your chosen site/campaign

A `workflow` is a wrapper containing methods to access all the processing steps.
For more information on the `WorkflowHandler` class, check out our [ReadTheDocs](https://es-sfgtools.readthedocs.io/en/latest/api/es_sfgtools.workflows.workflow_handler.html) documentation. 

Survey metadata will be also be displayed below. 

In [ ]:
workflow = WorkflowHandler()

workflow.set_network_station_campaign(
    network_id=NETWORK,
    station_id=STATION,
    campaign_id=CAMPAIGN
)

workflow.midprocess_get_sitemeta()

for campaign in workflow.current_station_metadata.campaigns:
    if campaign.name == CAMPAIGN:
        surveys = campaign.surveys
        print(f"\nThe campaign {STATION} {campaign.name} contains {len(surveys)} survey patterns\n")
        for i, survey in enumerate(surveys):
            print(f" {i+1}. Survey Name: {survey.id}")
            print(f"     Survey Type: {survey.type.value} ({survey.notes if survey.notes is not None else ''})")
            print(f"     {survey.start} to {survey.end}")
            print(f"     Survey length: {survey.end - survey.start}\n")

## Prepare data for GARPOS Processing 

Optional Variables to include in `mid_process_prep_garpos()`
  * `site_metadata`: Site metadata or path to metadata file. If not provided, it will be loaded if available.
    * Default: None
  * `survey_id`: Survey identifier to process. If None, processes all surveys
    * Default: None
  * `custom_filters`: Custom filter settings for shot data preparation
    * Default: None
  * `override`: True/False. If true, re-prepares existing data.
    * Default: False
  * `write_intermetediate`: True/False, If true, writes intermediate files.
    * Default: False

A value error will be raised if site metadata is not loaded. 

*For more information on this fucntion, check out our [ReadTheDocs](https://es-sfgtools.readthedocs.io/en/latest/api/es_sfgtools.workflows.workflow_handler.html#es_sfgtools.workflows.workflow_handler.WorkflowHandler.midprocess_prep_garpos)*

#### **Optional** 
Set up custom filters in the dictionary provided in the cell below.  These filters will reduce the number of samples being fed into GARPOS

Optional Filters
  * Pride Residiuals - Filter based on GNSS positioning residuals
    * Defaults
      * Disabled
      * Max Residual - 8mm if enabled
* Max distance from center - Filter shots beyond maximum distance from array center. This is only applied to center surveys whether enabled or diabled. 
  * Defaults
    * **Enabled**
    * Max distance in meters - 150
* Ping Replies - Filter based on minimum acoustic ping replies. 
  * Defaults
    * Disabled
    * Minimum replies - 1 
* Acoustic filters - Apply standard acoustic data quality filters"
  * Defaults
    * Disabled
    * Level - OK
      * `GOOD`: High quality data - SNR ≥ 20, DBV between -26 and -3, XC ≥ 60
      * `OK`: Medium quality data - SNR between 12-20, DBV between -36 and -3, XC between 45-60
      * `DIFFICULT`: Low quality data - SNR < 12, DBV < -36 or > -3, XC < 45
  


In [ ]:
# OPTIONAL: Edit the filter config as needed
FILTER_CONFIG = {
    "pride_residuals": {
        "enabled": False,
        "max_residual_mm": 8,
    },
    "max_distance_from_center": {
        "enabled": True,
        "max_distance_m": 150.0,
    },
    "ping_replies": {
        "enabled": False,
        "min_replies": 1,
    },
    "acoustic_filters": {
        "enabled": False,
        "level": "OK",
        "description": "Apply standard acoustic data quality filters",
    },
}

### Set up survey ID if running only 1 survey at a time
Use as a variable below in the `workflow.midprocess_prep_garpos()` function. Default is to run all surveys available.

In [ ]:
SURVEY_NUMBER = 1
SURVEY_ID=f"{CAMPAIGN}_{SURVEY_NUMBER}"

### Run the prep function

If you added custom filters above, add `custom_filters=FILTER_CONFIG` to the function below.  Custom filters are not required.

By default..
* Filters are all disabled, except for center point surveys the max_distance_m defaults to 150. 
* Runs all surveys in campaign, if you wish to only run 1 survey, add the `survey_id` as a variable in the `midprocess_prep_garpso` function

In [ ]:
# Prepare GARPOS data, insert custom filters if desired or other variables described above
workflow.midprocess_prep_garpos() # survey_id="", custom_filters=FILTER_CONFIG)

### Plot the preprocessed data completeness

This plot shows the ping/reply completeness for each transponder for each survey in the campaign.  It is plotted as % of expected replies in a 10 min bin.
It also shows any data removed by the above filters.


In [ ]:
workflow.modeling_plot_shotdata_replies_per_transponder()

## Run Garpos
This will loop through all the surveys in the campaign and run GARPOS on each.  

You can change the above inversion parameters and run again to compare results, but make sure to increment run_id so that the results are stored in a new directory.


Optional Variables to include in `modeling_run_garpos()`
  * `survey_id`: Optional survey identifier to process. If None, processes all surveys
    * Default: None
  * `run_id`: Identifier for the GARPOS run 
    * Default: "Test"
  * `iterations`: Number of GARPOS iterations to perform.  Multiple iterations can improve results, but increases processing runtime.
    * Default: 1  
  * `site_metadata`: Site metadata or local path to metadata file. If not provided, available metadata will be loaded from the archive.
    * Default: None
  * `override`: True/False. If true, re-runs GARPOS even if results exist.
    * Default False
  * `custom_settings`: Custom GARPOS settings to apply
    * GARPOS settings
        * Inversion Parameters - for more information: https://github.com/s-watanabe-jhod/garpos/blob/master/sample/Settings-prep.ini
            * `spline_degree` Default: 3
            * `log_lambda`  Smoothness parameter for background perturbation (in log10 scale) Default: [0]
            * `log_gradlambda` Smoothness parameter for spatial gradient ( = Lambda0 * gradLambda ) Default: -1
            * `mu_t`  Correlation length of data for transmit time (in min.) Default: [0.0]
            * `mu_mt`  Data correlation coefficient b/w the different transponders. Default: [0.5]
            * `knotint0` Typical Knot interval (in min.) for gamma's component (a0, a1, a2). Note ;; shorter numbers recommended, but consider the computational resources.  If knotint = 0, relevant component is fixed to be zero. Default: 5
            * `knotint1` Typical Knot interval (in min.) for gamma's component (a0, a1, a2). Note ;; shorter numbers recommended, but consider the computational resources.  If knotint = 0, relevant component is fixed to be zero. Default: 0
            * `knotint2` Typical Knot interval (in min.) for gamma's component (a0, a1, a2). Note ;; shorter numbers recommended, but consider the computational resources.  If knotint = 0, relevant component is fixed to be zero. Default: 0
            * `rejectcriteria`  Criteria for the rejection of data (+/- rsig * Sigma). if = 0, no data will be rejected during the process. This parameter should be 0 when applying ABIC search. Default: 2
            * `inversiontype`  Default: positions Options: positions, gammas, both
            * `positionalOffset` Positional offset for the inversion. Default: [0.0, 0.0, 0.0]
            * `traveltimescale`  Typical measurement error for travel time.  (= 1.e-4 sec is recommended in 10 kHz carrier) Default: 1.0e-4
            * `maxloop`  Maximum loop for iteration. Default: 50
            * `convcriteria`  Convergence criteria for model parameters. Default: 5.0e-3
            * `deltap` Infinitesimal values to make Jacobian matrix. Default: 1.0e-6
            * `deltab`  Infinitesimal values to make Jacobian matrix. Default: 1.0e-6
            * `delta_center_position` Delta Center Position. Default: 1, 1, 1

  *For more information on this function, check out our [ReadTheDocs](https://es-sfgtools.readthedocs.io/en/latest/api/es_sfgtools.workflows.workflow_handler.html#es_sfgtools.workflows.workflow_handler.WorkflowHandler.modeling_run_garpos)*

### Set up any custom settings


In [ ]:
# Example custom settings for GARPOS inversion
GARPOS_CUSTOM_SETTINGS = {
    'inversion_params' : {
        'maxloop': 5 # Maximum number of inversion loops
        }
    }

### Set up run ID and survey (if needed)

In [ ]:
SURVEY_NUMBER = 1
SURVEY_ID=f"{CAMPAIGN}_{SURVEY_NUMBER}"

RUN_ID = "1"   # Change name or number for each GARPOS run to avoid overwriting previous results

### Run garpos with options outlined above

In [ ]:
workflow.modeling_run_garpos(run_id=RUN_ID, 
                             survey_id=SURVEY_ID, 
                             custom_settings=GARPOS_CUSTOM_SETTINGS)

## Plot Results

Plot results from GARPOS run(s). 

Optional Parameters
  * `survey_id`: ID of the survey to plot results for
    * Default: None
  * `run_id`: The run ID of the survey results to plot
    * Default: "test"
  * `res_filter`: The residual filter value to filter outrageous values (m).
    * Default: 10
  * `save_fig`: True/False. If true, save the figure created.
    * Default: True
  * `show_fig`: True/False. If true, display figure.
    * Default True

  *For more information on this function, check out our [ReadTheDocs](https://es-sfgtools.readthedocs.io/en/latest/api/es_sfgtools.workflows.workflow_handler.html#es_sfgtools.workflows.workflow_handler.WorkflowHandler.modeling_plot_garpos_results)*


In [ ]:
workflow.modeling_plot_garpos_results(run_id=RUN_ID)

### Plot just the residuals from garpos in their own plot

In [ ]:
#include all data provided as input, and show the removed (flagged) data in blue and the data kept (unflagged) in orange
workflow.modeling_plot_flagged_residuals(run_id=RUN_ID) 

In [ ]:
#Show the residuals for only the data used in the solution
workflow.modeling_plot_garpos_residuals(run_id=RUN_ID) 